# Text Generation with Keras
This notebook explores how to train a Keras model to generate text in the style of William Shakespeare

In [9]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Let's load the text file containing all of Shakespeare's works. Check out Project Gutenburg to access the full text.

In [10]:
path = 'shakespeare.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 5573152


Now let's cut the text up into 40 character sequences to make it simpler to build sequences for training

In [11]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 79
nb sequences: 1857704


Now lets vectorize this

In [12]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Now we can build our model - note that the layer here we are using is LSTM, a form of Recurrent Neural Network

In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Let's define a function for sampling predictions at the end of each epoch

In [14]:
def sampler(prediction, temperature):
    prediction = np.asarray(prediction).astype('float64')
    prediction = np.log(prediction) / temperature
    exp_prediction = np.exp(prediction)
    final_pred = exp_prediction / np.sum(exp_prediction)
    prob = np.random.multinomial(1, final_pred, 1)
    return np.argmax(prob)

Let's define a function that allows us, at the end of each epoch, to output a sample of what the model has learned.

In [15]:
def on_epoch_end(epoch, _):
    print()
    print('Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sampler(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=10000,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
1857704/1857704 [==============================] - 1166s 628us/step - loss: 2.5464

Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "
it blesseth him that gives and him that"

it blesseth him that gives and him that the wird he wer the do the come the come the come the dome ser and the for her the so her be the prowe the have the will be the the wis and so ser so dest of the word of her the bear and the deen so me the sore the so and a do she so mand her a dous and the so dest in the will the be the will the will i                                                                                               
----- diversity: 0.5
----- Generating with seed: "
it blesseth him that gives and him that"

it blesseth him that gives and him that the me the bear, and me the come, and bet gone of lase of why my the mare?

                                                                                                                                    

    the proceated the free to the perfecting bear.

 [_exit._]

scene iii. the lady. if i will not the bear the parts,
----- diversity: 0.5
----- Generating with seed: "n in the blazon of sweet beauty’s best,
"
n in the blazon of sweet beauty’s best,
come of the dead man, and in the forest stand too,
    i shall say that the tame is contraring to dear mind,
    but that i have with the bardon to such a taunting of our frame.
  costard. hears the fair friend to the favour to castle.

berter.
i know not thee may not but the man in the king of boy.

king.
poor gods, macketer cares nature, the fire
    the chance and make the took of the books,
  
----- diversity: 1.0
----- Generating with seed: "n in the blazon of sweet beauty’s best,
"
n in the blazon of sweet beauty’s best,
or courtey which and pleamy.

ricades.
wes you gentlemen,
    how, my fearle attenmer, for and present,
what foech toby unnease the mont tairio:
do suboner for thine!-glessing again, for that her wight,
    in the va

Let's save the model

In [17]:
model.save('shakespeare_generator.h5')